In [2]:
import sys
def extract_cover_faces(input_file, output_file, target_material="Cover"):
    vertices = []
    texture_coords = []
    normals = []
    faces = []
    current_material = None
    used_vertex_indices = set()
    used_texcoord_indices = set()
    used_normal_indices = set()

    with open(input_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue

        prefix = parts[0]

        # 记录顶点、纹理坐标和法线
        if prefix == 'v':
            vertices.append(line.strip())
        elif prefix == 'vt':
            texture_coords.append(line.strip())
        elif prefix == 'vn':
            normals.append(line.strip())

        # 检查材质名称
        elif prefix == 'usemtl':
            current_material = parts[1]

        # 记录使用目标材质的面
        elif prefix == 'f' and current_material == target_material:
            face = parts[1:]
            faces.append(face)

            # 提取面中所有使用到的顶点、纹理坐标和法线的索引
            for vertex in face:
                indices = vertex.split('/')
                vertex_index = int(indices[0])
                used_vertex_indices.add(vertex_index)

                if len(indices) > 1 and indices[1]:
                    texcoord_index = int(indices[1])
                    used_texcoord_indices.add(texcoord_index)

                if len(indices) > 2 and indices[2]:
                    normal_index = int(indices[2])
                    used_normal_indices.add(normal_index)

    # 重新编号顶点、纹理坐标和法线索引
    vertex_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_vertex_indices), start=1)}
    texcoord_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_texcoord_indices), start=1)}
    normal_index_mapping = {old_index: new_index for new_index, old_index in enumerate(sorted(used_normal_indices), start=1)}

    # 构建新的顶点、纹理坐标和法线列表
    new_vertices = [vertices[old_index - 1] for old_index in sorted(used_vertex_indices)]
    new_texture_coords = [texture_coords[old_index - 1] for old_index in sorted(used_texcoord_indices)]
    new_normals = [normals[old_index - 1] for old_index in sorted(used_normal_indices)]

    # 更新面中的索引
    new_faces = []
    for face in faces:
        new_face = []
        for vertex in face:
            indices = vertex.split('/')
            new_vertex_index = str(vertex_index_mapping[int(indices[0])])

            new_texcoord_index = indices[1]
            if len(indices) > 1 and indices[1]:
                new_texcoord_index = str(texcoord_index_mapping[int(indices[1])])

            new_normal_index = indices[2]
            if len(indices) > 2 and indices[2]:
                new_normal_index = str(normal_index_mapping[int(indices[2])])

            new_indices = [new_vertex_index]
            if len(indices) > 1:
                new_indices.append(new_texcoord_index)
            if len(indices) > 2:
                new_indices.append(new_normal_index)

            new_face.append('/'.join(new_indices))
        new_faces.append(f'{" ".join(new_face)}')

    # 写入新的 obj 文件
    with open(output_file, 'w') as out:
        out.write("# Extracted OBJ file with Cover material\n")
        for vertex in new_vertices:
            out.write(f"{vertex}\n")
        for texcoord in new_texture_coords:
            out.write(f"{texcoord}\n")
        for normal in new_normals:
            out.write(f"{normal}\n")

        out.write(f"usemtl {target_material}\n")
        for face in new_faces:
            out.write(f"f {face}\n")
# 示例用法
if __name__ == "__main__":
    input_file = r"D:\study\Computer Graphics\Sparkium-v2\assets\mesh\bed\bed.obj"    # 输入的 OBJ 文件路径
    output_file = r"D:\study\Computer Graphics\Sparkium-v2\assets\mesh\bed\bed2.obj"  # 输出的 OBJ 文件路径
    extract_cover_faces(input_file, output_file)
    print(f"提取完成，结果已保存到 {output_file}")

提取完成，结果已保存到 D:\study\Computer Graphics\Sparkium-v2\assets\mesh\bed\bed2.obj
